In [138]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [139]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [140]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-05 07:00:00+00:00


In [141]:
import pandas as pd
import hopsworks  # Import Hopsworks for manual connection

HOPSWORKS_API_KEY = "11Pnc4YExMWVAIq0.11oLgfRGa7dwpPvQkiu6SA0zwpSAhSQjJ0mSgLdg2wKemRB7roS6culWyM51O3x6"
HOPSWORKS_PROJECT_NAME = "nyc_taxi_proj"

# Manually specify Feature Group Name
FEATURE_GROUP_MODEL_PREDICTION = "taxi_hourly_model_prediction"  # Adjust this if needed
FEATURE_GROUP_VERSION = 1  # Change based on your setup

def fetch_predictions():
    # Get the current timestamp (rounded to the next hour)
    current_date = pd.Timestamp.now(tz="Etc/UTC").ceil("h")

    # Connect to Hopsworks manually
    project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY, project=HOPSWORKS_PROJECT_NAME)
    fs = project.get_feature_store()

    # Get the feature group manually (no config)
    fg = fs.get_feature_group(
        name=FEATURE_GROUP_MODEL_PREDICTION,
        version=FEATURE_GROUP_VERSION
    )

    # Query to fetch predictions for the current hour
    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_date)

    return query.read()


In [142]:
# def fetch_predictions():
#     current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

#     fs = get_feature_store()
#     fg = fs.get_feature_group(
#         name=config.FEATURE_GROUP_MODEL_PREDICTION,
#         version=1
#     )

#     query = fg.select_all()
#     query = query.filter(fg.pickup_hour == current_hour)

#     return query.read()

In [143]:
import pandas as pd
import hopsworks  
HOPSWORKS_API_KEY = "11Pnc4YExMWVAIq0.11oLgfRGa7dwpPvQkiu6SA0zwpSAhSQjJ0mSgLdg2wKemRB7roS6culWyM51O3x6"
HOPSWORKS_PROJECT_NAME = "nyc_taxi_proj"
FEATURE_GROUP_MODEL_PREDICTION = "taxi_hourly_model_prediction"  # Adjust this if needed
FEATURE_GROUP_VERSION = 1  # Change based on your setup

# Connect to Hopsworks
project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY, project=HOPSWORKS_PROJECT_NAME)
fs = project.get_feature_store()  # Get feature store manually

# Get the feature group (NO config dependency)
fg = fs.get_feature_group(
    name=FEATURE_GROUP_MODEL_PREDICTION,
    version=FEATURE_GROUP_VERSION
)

# Query the entire feature group
query = fg.select_all()


2025-03-05 01:11:03,344 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 01:11:03,347 INFO: Initializing external client
2025-03-05 01:11:03,347 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 01:11:03,927 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665


In [144]:
# fs = get_feature_store()
# fg = fs.get_feature_group(
#     name=config.FEATURE_GROUP_MODEL_PREDICTION,
#     version=1
# )

# query = fg.select_all()

In [145]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (13.86s) 


In [146]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,229,86.0,2025-03-05 00:00:00+00:00
1,85,0.0,2025-03-05 00:00:00+00:00
2,36,0.0,2025-03-05 00:00:00+00:00
3,252,0.0,2025-03-05 00:00:00+00:00
4,133,0.0,2025-03-05 00:00:00+00:00
...,...,...,...
1752,163,92.0,2025-03-05 07:00:00+00:00
1753,213,1.0,2025-03-05 07:00:00+00:00
1754,261,7.0,2025-03-05 07:00:00+00:00
1755,48,88.0,2025-03-05 07:00:00+00:00


In [147]:
#a.info()

In [148]:
from datetime import datetime, timezone, timedelta
import pandas as pd
import hopsworks 
HOPSWORKS_API_KEY = "11Pnc4YExMWVAIq0.11oLgfRGa7dwpPvQkiu6SA0zwpSAhSQjJ0mSgLdg2wKemRB7roS6culWyM51O3x6"
HOPSWORKS_PROJECT_NAME = "nyc_taxi_proj"

FEATURE_GROUP_MODEL_PREDICTION = "taxi_hourly_model_prediction"
FEATURE_GROUP_VERSION = 1  

def fetch_next_hour_predictions():
    # Get current UTC time and round up to the next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    # Format as YYYY-MM-DD and YYYY-MM-DD HH:00
    current_date = next_hour.strftime("%Y-%m-%d")
    next_date = (next_hour + timedelta(days=1)).strftime("%Y-%m-%d")
    hour_str = next_hour.strftime("%Y-%m-%d %H:00")  # Format for filtering

    # Connect to Hopsworks
    project = hopsworks.login(api_key_value=HOPSWORKS_API_KEY, project=HOPSWORKS_PROJECT_NAME)
    fs = project.get_feature_store()

    # Get the feature group
    fg = fs.get_feature_group(name=FEATURE_GROUP_MODEL_PREDICTION, version=FEATURE_GROUP_VERSION)

    # Query feature group for today's data
    df = fg.select_all().read()

    # Convert 'pickup_hour' to datetime if it's not already
    df["pickup_hour"] = pd.to_datetime(df["pickup_hour"])

    # Filter for the next hour
    df_hour = df[df["pickup_hour"] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")

    return df_hour

# Example usage
predictions = fetch_next_hour_predictions()
print(predictions)


2025-03-05 01:11:20,220 INFO: Closing external client and cleaning up certificates.


Connection closed.
2025-03-05 01:11:20,222 INFO: Initializing external client
2025-03-05 01:11:20,222 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 01:11:20,822 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.37s) 
Current UTC time: 2025-03-05 06:11:20.220072+00:00
Next hour: 2025-03-05 07:00:00+00:00
Querying for date range: 2025-03-05 to 2025-03-06
Filtering for hour: 2025-03-05 07:00
Found 251 records
      pickup_location_id  predicted_demand               pickup_hour
1506                  71               1.0 2025-03-05 07:00:00+00:00
1507                 130               1.0 2025-03-05 07:00:00+00:00
1508                  43              69.0 2025-03-05 07:00:00+00:00
1509                  14               1.0 2025-03-05 07:00:00+00:00
1510                  42               4.0 2025-03-05 07:00:00+00:00
...                  

In [149]:
# from datetime import datetime, timezone, timedelta

# def fetch_next_hour_predictions():
#     # Get current UTC time and round up to next hour
#     now = datetime.now(timezone.utc)
#     next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

#     # Extract components from the rounded-up time
#     year = next_hour.year
#     month = next_hour.month
#     day = next_hour.day
#     hour = next_hour.hour

#     # Create date strings in YYYY-MM-DD format
#     current_date = f"{year}-{month:02d}-{day:02d}"
#     next_date = next_hour + timedelta(days=1)
#     next_date_str = next_date.strftime("%Y-%m-%d")

#     fs = get_feature_store()
#     fg = fs.get_feature_group(
#         name=config.FEATURE_GROUP_MODEL_PREDICTION,
#         version=1
#     )

#     # # First get the full day's data from Hopsworks
#     # df = fg.filter(
#     #     (fg.pickup_hour >= current_date) &
#     #     (fg.pickup_hour < next_date_str)
#     # ).read()

#     # Then filter for next hour in the DataFrame
#     hour_str = f"{current_date} {hour:02d}:00"
#     df_hour = df[df['pickup_hour'] == hour_str]

#     print(f"Current UTC time: {now}")
#     print(f"Next hour: {next_hour}")
#     print(f"Querying for date range: {current_date} to {next_date_str}")
#     print(f"Filtering for hour: {hour_str}")
#     print(f"Found {len(df_hour)} records")
#     return df_hour

# # Example usage:
# predictions = fetch_next_hour_predictions()

In [150]:
now = datetime.now(timezone.utc)

In [151]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
1506,71,1.0,2025-03-05 07:00:00+00:00
1507,130,1.0,2025-03-05 07:00:00+00:00
1508,43,69.0,2025-03-05 07:00:00+00:00
1509,14,1.0,2025-03-05 07:00:00+00:00
1510,42,4.0,2025-03-05 07:00:00+00:00
...,...,...,...
1752,163,92.0,2025-03-05 07:00:00+00:00
1753,213,1.0,2025-03-05 07:00:00+00:00
1754,261,7.0,2025-03-05 07:00:00+00:00
1755,48,88.0,2025-03-05 07:00:00+00:00


In [152]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1757 entries, 0 to 1756
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  1757 non-null   int32                  
 1   predicted_demand    1757 non-null   float64                
 2   pickup_hour         1757 non-null   datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 34.4 KB


In [153]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [154]:
dt = current_date.ceil('h')


In [155]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-05 01:11:33,048 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 01:11:36,056 INFO: Initializing external client
2025-03-05 01:11:36,056 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 01:11:36,593 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.57s) 


In [156]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.44s) 


,pickup_location_id,predicted_demand,pickup_hour


In [157]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,229,86.0,2025-03-05 00:00:00+00:00
1,85,0.0,2025-03-05 00:00:00+00:00
2,36,0.0,2025-03-05 00:00:00+00:00
3,252,0.0,2025-03-05 00:00:00+00:00
4,133,0.0,2025-03-05 00:00:00+00:00
...,...,...,...
1752,163,92.0,2025-03-05 07:00:00+00:00
1753,213,1.0,2025-03-05 07:00:00+00:00
1754,261,7.0,2025-03-05 07:00:00+00:00
1755,48,88.0,2025-03-05 07:00:00+00:00


In [158]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [159]:
predictions = fetch_next_hour_predictions()

2025-03-05 01:11:39,705 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 01:11:39,707 INFO: Initializing external client
2025-03-05 01:11:39,707 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 01:11:40,353 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665
Current UTC time: 2025-03-05 06:11:39.705121+00:00
Next hour: 2025-03-05 07:00:00+00:00
Found 251 records


In [160]:
fs = get_feature_store()
fg = fs.get_feature_group(name=FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-05 01:11:41,300 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 01:11:41,303 INFO: Initializing external client
2025-03-05 01:11:41,303 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 01:11:41,853 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214665


In [161]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.39s) 


In [162]:
df["pickup_hour"].max()

Timestamp('2025-03-05 07:00:00+0000', tz='Etc/UTC')

In [163]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-05 07:00:00+00:00


In [164]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
1506,71,1.0,2025-03-05 07:00:00+00:00
1507,130,1.0,2025-03-05 07:00:00+00:00
1508,43,69.0,2025-03-05 07:00:00+00:00
1509,14,1.0,2025-03-05 07:00:00+00:00
1510,42,4.0,2025-03-05 07:00:00+00:00
...,...,...,...
1752,163,92.0,2025-03-05 07:00:00+00:00
1753,213,1.0,2025-03-05 07:00:00+00:00
1754,261,7.0,2025-03-05 07:00:00+00:00
1755,48,88.0,2025-03-05 07:00:00+00:00
